<a href="https://colab.research.google.com/github/ocalru/EDP-I-Simulacion-I/blob/main/Proyecto_Simulacion_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install simpy

import simpy
import random
from statistics import mean

# =====================
# Parámetros
# =====================
RANDOM_SEED = 42
LAMBDA = 4        # tasa de llegadas λ
MU = 6            # tasa de servicio μ (por servidor)
SERVERS = 3       # >= 3 servidores
SIM_TIME = 1000   # tiempo de simulación

# =====================
# Datos a recolectar
# =====================
W = []    # tiempo en sistema
Wq = []   # tiempo en cola

# Para L (promedio de N en el sistema) vía área bajo N(t)
N = 0                 # número en el sistema
t_prev = 0.0          # último tiempo en que N cambió
AreaN = 0.0           # integral N(t) dt

def actualizar_N(env, nuevoN):
    """Acumula área bajo N(t) y actualiza el contador N cuando hay llegada/salida."""
    global N, t_prev, AreaN
    t = env.now
    dt = t - t_prev
    AreaN += dt * N
    N = nuevoN
    t_prev = t

# =====================
# Procesos
# =====================
def cliente(env, servidores):
    llegada = env.now
    actualizar_N(env, N + 1)  # entra al sistema

    with servidores.request() as req:
        yield req
        inicio_servicio = env.now
        Wq.append(inicio_servicio - llegada)

        yield env.timeout(random.expovariate(MU))  # servicio ~ Exp(MU)

    salida = env.now
    W.append(salida - llegada)

    actualizar_N(env, N - 1)  # sale del sistema

def generador(env, servidores):
    while True:
        yield env.timeout(random.expovariate(LAMBDA))  # interarribo ~ Exp(LAMBDA)
        env.process(cliente(env, servidores))

# =====================
# Simulación
# =====================
random.seed(RANDOM_SEED)

env = simpy.Environment()
servidores = simpy.Resource(env, capacity=SERVERS)

env.process(generador(env, servidores))
env.run(until=SIM_TIME)

# Cerrar el último tramo para L
dt = env.now - t_prev
AreaN += dt * N

L = AreaN / env.now

# =====================
# Resultados
# =====================
print("RESULTADOS")
print("----------")
print("Servidores:", SERVERS)
print("W  promedio (tiempo en sistema):", round(mean(W), 4))
print("Wq promedio (tiempo en cola):   ", round(mean(Wq), 4))
print("L  promedio (clientes en sistema):", round(L, 4))






RESULTADOS
----------
Servidores: 3
W  promedio (tiempo en sistema): 0.1672
Wq promedio (tiempo en cola):    0.0017
L  promedio (clientes en sistema): 0.6686
